In [8]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [4]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import VanGame.utils as utils
import scipy as sp
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [9]:
sequence = sorted([p for p in CELLS])
sequence

[(-3, 0),
 (-3, 1),
 (-3, 2),
 (-3, 3),
 (-2, -1),
 (-2, 0),
 (-2, 1),
 (-2, 2),
 (-2, 3),
 (-1, -2),
 (-1, -1),
 (-1, 0),
 (-1, 1),
 (-1, 2),
 (-1, 3),
 (0, -3),
 (0, -2),
 (0, -1),
 (0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, -3),
 (1, -2),
 (1, -1),
 (1, 0),
 (1, 1),
 (1, 2),
 (2, -3),
 (2, -2),
 (2, -1),
 (2, 0),
 (2, 1),
 (3, -3),
 (3, -2),
 (3, -1),
 (3, 0)]

In [6]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(100,activation='relu', input_shape=(11,)))
    model.add(layers.Dense(200, activation='relu'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [7]:
def build_linear():
    model = models.Sequential()
    model.add(layers.Dense(1,activation='linear', input_shape=(11,)))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [6]:
model = build_linear()

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [11]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [12]:
def load_logs():
    datapath = join(os.getcwd(),'rec/')
    van_files = bash_call(['ls', datapath])
    for i in range(len(van_files)):
        van_files[i] = 'rec/' + van_files[i]
    
    datapath = join(os.getcwd(),'hardcode_rec/')
    hardcode_file = bash_call(['ls', datapath])
    
    for i in range(len(hardcode_file)):
        hardcode_file[i] = 'hardcode_rec/' + hardcode_file[i]

    return van_files + hardcode_file

In [13]:
import math

def sigmoid(x):
    return 1/(1+math.exp(-x))

In [14]:
def PG_log(filename, condition=None, value_check=False):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    #print(filename)
    states, color = utils.load_l(filename)
    
    train = []
    value = []
    
    for s in states:
        # print(s)
        board = []
        # add board
        for cord in sequence:
            # does not include the board
            break
            board.append(s['board'][cord])
        # color 
        # board.append(color)
        board.append(s['turns'])
        # [hueristic difference, piece difference, danger piece, 
        # (red, blue green) exit piece with rotate, (red, blue green) heuristic with rotate]
        board += s['utility']
        # calculate based on (hueristic difference, piece difference, danger piece)
        board.append(s['ev'])
        train.append(board)
        value.append(s['rew'])
        
    if value_check:
        print(value)
        
    value.reverse()
    decay = 0.8
    curr_decay = decay
    for i in range(1, len(value)):
        value[i] += value[i-1] * curr_decay
        curr_decay *= decay
        value[i] = round(value[i], 3)
        
        if value[i] < 0.0001:
            value[i] = 0
    
    value.reverse()
    
    if value_check:
        print(value)
    
    # post_value = np.array(value)
    
    # post_value = np.tanh(post_value)
    
    # normalize the value
    # post_value = (post_value - post_value.mean()) / post_value.std()
    
    if value_check:
        print("\n",post_value)
    
    return train, value

In [16]:
# this is only for test, don't run
t,v = PG_log('hardcode_rec/[1win]1557044414.0caoredjiba.txt', value_check=False)
print(v)

[0, 0, 0, 0.004, 1.005, 1.006, 1.0, 0.047, 5.06, 5.216, 15.0, 0, 0, 0.009, 0.242, 5.501, 100.064, 0.927, 10.789, 100.477, 3.551, 21.165, 100.922, 3.518, 10.737, 26.214, 51.2, 80.0, 100]


In [1]:
!ls 'hardcode_rec'

'[1win]1557044414.0caoredjiba.txt'    '[2zzz]1557044245.0caoredjiba.txt'
'[1win]1557044506.0caoredjiba.txt'    '[2zzz]1557044286.0caoredjiba.txt'
'[1win]1557046125.0caogreenjiba.txt'  '[2zzz]1557046125.0caobluejiba.txt'
'[1win]1557046150.0caogreenjiba.txt'  '[2zzz]1557046125.0caoredjiba.txt'
'[1win]1557046182.0caogreenjiba.txt'  '[2zzz]1557046150.0caobluejiba.txt'
'[1win]1557046561.0caogreenjiba.txt'  '[2zzz]1557046150.0caoredjiba.txt'
'[1win]1557046648.0caoredjiba.txt'    '[2zzz]1557046182.0caobluejiba.txt'
'[1win]1557047301.0caoredjiba.txt'    '[2zzz]1557046182.0caoredjiba.txt'
'[1win]1557047724.0caoredjiba.txt'    '[2zzz]1557048236.0caoredjiba.txt'
'[1win]1557048236.0caobluejiba.txt'   '[2zzz]1557048369.0caoredjiba.txt'
'[1win]1557048369.0caobluejiba.txt'   '[2zzz]1557048513.0caoredjiba.txt'
'[1win]1557048513.0caogreenjiba.txt'  '[2zzz]1557065138.0caogreenjiba.txt'
'[1win]1557054394.0caoredjiba.txt'    '[2zzz]1557065439.0caobluejiba.txt'
'[1win]1557065574.0caogreenjiba.txt'  '[2zzz]

In [13]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')

In [14]:
def save_model():
    model.save('VanGame/my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('VanGame/trained_model', 'wb+') as f:
        pickle.dump(model.get_weights(), f)

In [15]:
def train(epochs, save=True):
    logfs = load_logs()
    train_set = []
    target_set = []
    
    for logf in logfs:
        train, value = PG_log(logf)
        if train:
            # print(train)
            # print(len(train), len(value))
            #print(train[-1])
            train_set += train
            # the target shouldnt be calculate like this, should use bootstrapping
            target_set += value

    model.fit(np.array(train_set), target_set, epochs=epochs, batch_size=1)
    
    if save:
        print("saved")
        save_model()

In [12]:
# don't run this, or ask me
#save_model()
#del model

try:
    model = load_model('VanGame/my_model.h5')
except:
    print("none model found")
    model = build_model()

Instructions for updating:
Use tf.cast instead.


In [29]:
def full_train_cycle(n, epochs=2):
    for i in range(n):
        !python3 -m referee HardCode VanGame VanGame -l logtry -v 0
    if ( len(bash_call(['ls', 'rec'])) + len(bash_call(['ls', 'hardcode_rec'])) ) > 0:
        train(epochs=epochs)
        !rm -v ./rec/*
        !rm -v ./hardcode_rec/*

In [19]:
for i in range(8000):
    print("********** starting round", i, "**********")
    full_train_cycle(1, epochs=2)

********** starting round 0 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
62/62 [==============================] - 0s 3ms/step - loss: 2536.7780
Epoch 2/2
62/62 [==============================] - 0s 3ms/step - loss: 2259.1307
saved
removed './rec/[2zzz]1556811097.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556811097.0caogreenjiba.txt'
removed './hardcode_rec/[2zzz]1556811097.0caobluejiba.txt'
********** starting round 1 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
74/74 [==============================] - 0s 3ms/step - loss: 3164.3734
Epoch 2/2
74/74 [==============================] - 0s 3ms/step - loss: 2940.9666
saved
removed './rec/[2zzz]1556811099.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556811099.0caogreenjiba.txt'
removed './hardcode_rec/[2zzz]1556811099.0caobluejiba.txt'
********** starting round 2 **********
* NOTE: unable to measure memory usage 

* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
102/102 [==============================] - 0s 3ms/step - loss: 1319.1047
Epoch 2/2
102/102 [==============================] - 0s 3ms/step - loss: 952.2943
saved
removed './rec/[2zzz]1556811144.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556811144.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556811144.0caogreenjiba.txt'
********** starting round 20 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/2
127/127 [==============================] - 0s 3ms/step - loss: 587.0978
Epoch 2/2
127/127 [==============================] - 0s 3ms/step - loss: 300.1164
saved
removed './rec/[1win]1556811147.0caoredjiba.txt'
removed './hardcode_rec/[2zzz]1556811147.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556811147.0caogreenjiba.txt'
********** starting round 21 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* wi

********** starting round 39 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
65/65 [==============================] - 0s 3ms/step - loss: 1895.5508
Epoch 2/2
65/65 [==============================] - 0s 3ms/step - loss: 1248.0270
saved
removed './rec/[2zzz]1556811186.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556811186.0caogreenjiba.txt'
removed './hardcode_rec/[2zzz]1556811186.0caobluejiba.txt'
********** starting round 40 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
90/90 [==============================] - 0s 3ms/step - loss: 2564.7525
Epoch 2/2
90/90 [==============================] - 0s 3ms/step - loss: 1617.1011
saved
removed './rec/[2zzz]1556811188.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556811188.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556811188.0caogreenjiba.txt'
********** starting round 41 **********
* NOTE: unable to measure memory usag

removed './hardcode_rec/[1win]1556811223.0caogreenjiba.txt'
removed './hardcode_rec/[2zzz]1556811223.0caobluejiba.txt'
********** starting round 57 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
90/90 [==============================] - 0s 3ms/step - loss: 1746.7124
Epoch 2/2
90/90 [==============================] - 0s 3ms/step - loss: 1360.7188
saved
removed './rec/[2zzz]1556811225.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556811225.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556811225.0caogreenjiba.txt'
********** starting round 58 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/2
79/79 [==============================] - 0s 3ms/step - loss: 441.5421
Epoch 2/2
79/79 [==============================] - 0s 3ms/step - loss: 374.3362
saved
removed './rec/[1win]1556811227.0caoredjiba.txt'
removed './hardcode_rec/[2zzz]1556811227.0caobluejiba.txt'
removed './hardcode_re

* winner: Green
Epoch 1/2
65/65 [==============================] - 0s 6ms/step - loss: 2475.4807
Epoch 2/2
65/65 [==============================] - 0s 3ms/step - loss: 2105.4170
saved
removed './rec/[2zzz]1556811265.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556811265.0caogreenjiba.txt'
removed './hardcode_rec/[2zzz]1556811265.0caobluejiba.txt'
********** starting round 76 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
87/87 [==============================] - 0s 5ms/step - loss: 706.1449
Epoch 2/2
87/87 [==============================] - 0s 3ms/step - loss: 308.5181
saved
removed './rec/[2zzz]1556811268.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556811268.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556811268.0caogreenjiba.txt'
********** starting round 77 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
99/99 [==============================] - 0s 3ms/step -

...

56/56 [==============================] - 0s 3ms/step - loss: 149.9479
Epoch 2/2
56/56 [==============================] - 0s 3ms/step - loss: 135.8447
saved
removed './rec/[2zzz]1556835197.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556835197.0caogreenjiba.txt'
removed './hardcode_rec/[2zzz]1556835197.0caobluejiba.txt'
********** starting round 7882 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* draw detected: game state occurred 4 times.
********** starting round 7883 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* draw detected: game state occurred 4 times.
********** starting round 7884 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* draw detected: game state occurred 4 times.
********** starting round 7885 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* draw detected: game state occurred 4 times.
********** starting round 7886 **********
* 

saved
removed './rec/[1win]1556835271.0caoredjiba.txt'
removed './hardcode_rec/[2zzz]1556835271.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556835271.0caogreenjiba.txt'
********** starting round 7908 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
72/72 [==============================] - 0s 4ms/step - loss: 219.4840
Epoch 2/2
72/72 [==============================] - 0s 3ms/step - loss: 152.7947
saved
removed './rec/[2zzz]1556835275.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556835275.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556835275.0caogreenjiba.txt'
********** starting round 7909 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
93/93 [==============================] - 0s 3ms/step - loss: 166.0782
Epoch 2/2
93/93 [==============================] - 0s 3ms/step - loss: 118.1179
saved
removed './rec/[2zzz]1556835277.0caoredjiba.txt'
removed './hardcode_rec

saved
removed './rec/[2zzz]1556835339.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556835339.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556835339.0caogreenjiba.txt'
********** starting round 7929 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
81/81 [==============================] - 0s 3ms/step - loss: 146.0201
Epoch 2/2
81/81 [==============================] - 0s 3ms/step - loss: 83.1171
saved
removed './rec/[2zzz]1556835341.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556835341.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556835341.0caogreenjiba.txt'
********** starting round 7930 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
65/65 [==============================] - 0s 5ms/step - loss: 43.6406
Epoch 2/2
65/65 [==============================] - 0s 3ms/step - loss: 32.1616
saved
removed './rec/[2zzz]1556835343.0caoredjiba.txt'
removed './hardcode_rec/[

108/108 [==============================] - 0s 3ms/step - loss: 158.7639
Epoch 2/2
108/108 [==============================] - 0s 3ms/step - loss: 85.4358
saved
removed './rec/[2zzz]1556835445.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556835445.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556835445.0caogreenjiba.txt'
********** starting round 7968 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
75/75 [==============================] - 0s 3ms/step - loss: 44.6399
Epoch 2/2
75/75 [==============================] - 0s 3ms/step - loss: 35.7434
saved
removed './rec/[2zzz]1556835447.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556835447.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556835447.0caogreenjiba.txt'
********** starting round 7969 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
87/87 [==============================] - 0s 3ms/step - loss: 120.3754
Epoch 2

removed './hardcode_rec/[2zzz]1556835510.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556835510.0caogreenjiba.txt'
********** starting round 7992 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Blue
Epoch 1/2
96/96 [==============================] - 0s 4ms/step - loss: 112.5033
Epoch 2/2
96/96 [==============================] - 0s 3ms/step - loss: 84.1900
saved
removed './rec/[2zzz]1556835513.0caoredjiba.txt'
removed './hardcode_rec/[1win]1556835513.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1556835513.0caogreenjiba.txt'
********** starting round 7993 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* draw detected: game state occurred 4 times.
********** starting round 7994 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
113/113 [==============================] - 0s 3ms/step - loss: 61.3500
Epoch 2/2
113/113 [==============================] - 0

In [30]:
for i in range(10):
    print("********** starting round", i+1, "**********")
    full_train_cycle(1, epochs=2)

********** starting round 1 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
175/175 [==============================] - 0s 939us/step - loss: 1405.6271
Epoch 2/2
175/175 [==============================] - 0s 998us/step - loss: 1405.5212
saved
removed './rec/[1win]1557044011.0caogreenjiba.txt'
removed './rec/[1win]1557044033.0caogreenjiba.txt'
removed './rec/[2zzz]1557044011.0caoredjiba.txt'
removed './rec/[2zzz]1557044033.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1557044011.0caobluejiba.txt'
removed './hardcode_rec/[2zzz]1557044033.0caoredjiba.txt'
********** starting round 2 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/2
85/85 [==============================] - 0s 974us/step - loss: 887.0895
Epoch 2/2
85/85 [==============================] - 0s 1ms/step - loss: 886.1126
saved
removed './rec/[2zzz]1557044036.0caobluejiba.txt'
removed './rec/[2zzz]1557044036.0caogree

In [18]:
model.get_weights()

[array([[ 1.0052652 ],
        [-0.85445476],
        [-6.684119  ],
        [-2.5960364 ],
        [ 2.4456909 ],
        [ 0.23364225],
        [ 0.8310305 ],
        [-1.6953737 ],
        [ 0.65038776],
        [ 0.8128255 ],
        [-0.1043197 ]], dtype=float32), array([0.81027883], dtype=float32)]

In [21]:
model.get_weights()

[array([[ 0.7907868 ],
        [-0.32162404],
        [-8.535148  ],
        [-3.3907456 ],
        [ 2.4803574 ],
        [-0.27139094],
        [ 0.21796495],
        [-1.6832609 ],
        [ 0.745602  ],
        [ 0.8406177 ],
        [-0.06827426]], dtype=float32), array([0.73863804], dtype=float32)]

In [23]:
model.get_weights()

[array([[  0.6433005 ],
        [ -0.42434895],
        [-10.670088  ],
        [ -4.065991  ],
        [  2.610408  ],
        [ -0.7354231 ],
        [ -0.31823027],
        [ -1.6468699 ],
        [  0.73553073],
        [  0.8307466 ],
        [ -0.12354679]], dtype=float32), array([0.7368741], dtype=float32)]

In [25]:
model.get_weights()

[array([[  0.7502025 ],
        [ -1.6158445 ],
        [-16.531923  ],
        [ -6.104451  ],
        [  3.544992  ],
        [ -0.8442374 ],
        [ -0.5856429 ],
        [ -1.9115182 ],
        [  0.73990107],
        [  0.9539051 ],
        [ -0.24140982]], dtype=float32), array([0.9105143], dtype=float32)]

In [27]:
# [hueristic difference, piece difference, danger piece, 
# (red, blue green) exit piece with rotate, (red, blue green) heuristic with rotate]
model.get_weights()

[array([[  0.65183425],
        [ -2.384349  ],
        [-22.84248   ],
        [ -8.487225  ],
        [  3.9976516 ],
        [ -0.93083483],
        [ -0.73155224],
        [ -2.0233183 ],
        [  0.71813434],
        [  1.1058966 ],
        [ -0.3911529 ]], dtype=float32), array([0.91981167], dtype=float32)]

# Linear Model

In [5]:
import sklearn

In [7]:
from sklearn.linear_model import SGDRegressor

In [42]:
!ls 'hardcode_rec'

'[1win]1557033792.0caoredjiba.txt'    '[2zzz]1557034206.0caobluejiba.txt'
'[1win]1557034206.0caoredjiba.txt'    '[2zzz]1557034539.0caoredjiba.txt'
'[1win]1557034539.0caogreenjiba.txt'  '[2zzz]1557034540.0caobluejiba.txt'
'[1win]1557035037.0caoredjiba.txt'    '[2zzz]1557035037.0caobluejiba.txt'
'[1win]1557035399.0red.txt'	      '[2zzz]1557035399.0blue.txt'
'[2zzz]1557033792.0caobluejiba.txt'


In [88]:
t,v = PG_log('hardcode_rec/[1win]1557034206.0caoredjiba.txt', value_check=False)
print(t)

[[1, -1, 0, 0, 0, 0, 0, 27, 28, 28, 8], [2, -2, 0, 0, 0, 0, 0, 25, 27, 27, 16], [3, -1, 0, 0, 0, 0, 0, 24, 27, 26, 8], [4, -2, 0, 0, 0, 0, 0, 22, 26, 25, 16], [5, -2, 0, 0, 0, 0, 0, 20, 25, 26, 16], [6, -1, 0, 0, 0, 0, 0, 19, 26, 24, 8], [7, -1, 0, 0, 0, 0, 0, 18, 25, 23, 8], [8, -1, 0, 0, 0, 0, 0, 17, 26, 22, 8], [9, -1, 0, 0, 0, 0, 0, 16, 26, 21, 8], [10, 0, 0, 1, 0, 0, 0, 16, 26, 20, -20], [11, -1, 0, 0, 0, 0, 0, 15, 26, 20, 8], [12, -1, 0, 0, 0, 0, 0, 14, 25, 20, 8], [13, -1, -1, 0, 1, 0, 0, 13, 27, 20, 3], [14, -1, 0, 0, 1, 0, 0, 12, 25, 19, 8], [15, -1, 0, 0, 1, 0, 0, 11, 24, 18, 8], [16, -2, 0, 1, 1, 0, 0, 9, 25, 16, -4], [17, -1, 0, 0, 1, 0, 0, 16, 21, 16, 2], [18, 0, 0, 0, 1, 0, 0, 16, 21, 14, 0], [19, 2, 1, 1, 1, 0, 0, 8, 24, 12, 6], [20, -1, 0, 0, 1, 0, 1, 15, 21, 11, 2], [21, -1, -1, 0, 2, 0, 1, 14, 21, 10, -18], [22, 4, 1, 1, 2, 0, 1, 8, 24, 9, 2], [23, 7, 1, 0, 2, 0, 1, 11, 21, 13, 6], [24, -1, 0, 0, 2, 0, 2, 10, 22, 12, 8], [25, -1, 0, 0, 2, 0, 2, 9, 22, 11, 8], [26, -1,

In [209]:
def train_linear(model, epochs):
    logfs = load_logs()
    train_set = []
    target_set = []
    
    for logf in logfs:
        train, value = PG_log(logf)
        if train:
            # print(train)
            # print(len(train), len(value))
            #print(train[-1])
            train_set += train
            # the target shouldnt be calculate like this, should use bootstrapping
            target_set += value

    model.fit(np.array(train_set), target_set, epochs=epochs, batch_size=1)
    
    return model

In [210]:
def build_linear():
    model = models.Sequential()
    model.add(layers.Dense(1,activation='linear', input_shape=(11,)))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [211]:
md = build_linear()

In [212]:
md = train_linear(md, 2)

IndexError: list index out of range

In [202]:
md.get_weights()

[array([[-0.27864483],
        [-0.2602383 ],
        [-0.8044047 ],
        [-0.07973181],
        [ 0.93857783],
        [ 0.4902317 ],
        [-0.08536516],
        [ 0.40277416],
        [ 0.34595937],
        [ 0.6034359 ],
        [-0.2099837 ]], dtype=float32), array([0.0861373], dtype=float32)]

In [163]:
reg = SGDRegressor(loss="squared_loss", penalty=None, verbose=1)

In [173]:
reg.partial_fit(t,v)

-- Epoch 1
Norm: 111999722346.67, NNZs: 10, Bias: 335019436.461521, T: 32, Avg. loss: 3178921561699423679938560.000000
Total training time: 0.00 seconds.


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty=None, power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=1, warm_start=False)

In [174]:
reg.coef_

array([-6.75056008e+10,  2.53010020e+10,  1.67605533e+10,  1.30503117e+10,
       -2.49022454e+10,  0.00000000e+00, -2.83062702e+10, -1.60451813e+10,
       -1.98059074e+10, -2.25164626e+10,  6.57075683e+10])

In [175]:
reg.predict([t[0]])

array([-1.18505729e+12])

# CNN attamp

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
md = Sequential()
#add model layers
md.add(Conv2D(1, kernel_size=3, activation='relu', input_shape=(28,28,1), use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
md.add(Flatten())
md.compile(optimizer='adam', loss='mse')

In [ ]:
for i in md.get_weights():
    print(i.shape)

In [ ]:
md.summary()

In [ ]:
md.predict(np.array([[[[1]*28]*28]]).reshape(1,28,28,1))

In [ ]:
asdf = md.get_weights()[0].reshape(3,3)

In [ ]:
wds = md.get_weights()

In [ ]:
convp = wds[0]

In [ ]:
convp.shape

In [ ]:
c = convp.reshape(1, 3, 3, 1)

In [ ]:
d = c[0].reshape(3,3)

In [ ]:
import scipy.signal as signal

In [ ]:
a = np.array([[[1]*28]*28]).reshape(28,28)

In [ ]:
e = signal.convolve2d(a, asdf, mode='valid')

In [ ]:
e

In [ ]:
signal.convolve(e, )

In [19]:
for i in range(10-1,0,-1):
    print(i)

9
8
7
6
5
4
3
2
1
